In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import layers
from keras.layers.experimental import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('../input/covid19globaldata/time_series_covid19_confirmed_global.csv') 

In [ ]:
data

In [ ]:
country = "India"

In [ ]:
data_confirmed = data[data["Country/Region"] == country]
data_confirmed

In [ ]:
data_confirmed = pd.DataFrame(data_confirmed[data_confirmed.columns[4:]].sum(),columns=["confirmed"])
len(data_confirmed)

In [ ]:
data_confirmed.index = pd.to_datetime(data_confirmed.index, format='%m/%d/%y')
len(data_confirmed)

In [ ]:
data_confirmed.plot(figsize=(10,5), title="confirmed_cases")
# data _confirmed = pd.to_datetime(data['Date'], infer_datetime_format=True)) 
# print(data['Date'])


In [ ]:
data_confirmed.describe()
# data.resample('A').mean().plot(subplots=True)

In [ ]:
x = len(data_confirmed) - 15

In [ ]:
train = data_confirmed.iloc[:x]
test = data_confirmed.iloc[x:]

In [ ]:
# train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
# train_features = train_dataset.copy()
# test_features = test_dataset.copy()

# train_labels = train_features.pop('Confirmed')
# test_labels = test_features.pop('Confirmed')
# print(train_dataset)

In [ ]:
# #normilizing ::
# normilizer = preprocessing.Normalization()
# normilizer.adapt(np.array(train_fdeatures))
# print(normilizer.mean.numpy())

# DATA SCALING ::
scaler = MinMaxScaler()
scaler.fit(train)

In [ ]:
train_scaled = scaler.transform(train)
test_scaled = scaler.transform(test)

In [ ]:
len(test_scaled)

In [ ]:
# features = data_scaled
# target = data_scaled[:,1]
# print(target)

In [ ]:
#### to find the next value
# TimeseriesGenerator(features, target, length=5, sampling_rate=1, batch_size=1)[0]

# print(ts_generator[0])

In [ ]:
win_length = 5
batch_size = 32
num_features = 1
train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(test_scaled, test_scaled, length=win_length, sampling_rate=1, batch_size=batch_size)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, activation = 'relu', input_shape=(win_length, num_features), return_sequences=True))
# model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
# model.add(tf.keras.layers.LSTM(128, return_sequences=True))
# model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
# model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.LSTM(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(32))
# model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1))


In [ ]:
model.summary()

In [ ]:
# print(tf.__version__)

In [ ]:
model.compile(loss=tf.losses.MeanSquaredError(),
             optimizer=tf.optimizers.Adam(),
             metrics=['accuracy'])

history = model.fit_generator(train_generator, epochs=50,
                              validation_data=test_generator,
                              steps_per_epoch=10)


In [ ]:
loss = history.history['loss']
val_loss =  history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'y', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.ylim([-0.2, 1])
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# model.evaluate_generator(test_generator, verbose=0)


In [ ]:
prediction = [] 

current_batch = train_scaled[-win_length:] #Final data points in train 
current_batch = current_batch.reshape(1, win_length, num_features) #Reshape
# current_pred = model.predict(current_batch)[0]
# prediction.append(current_pred)

In [ ]:
# len(current_batch)

In [ ]:
# prediction

In [ ]:
# predictions = model.predict_generator(test_generator)
future_days = 40
for i in range(len(test) + future_days):
    current_pred = model.predict(current_batch)[0]
    prediction.append(current_pred)
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
# to get actual numbers
rescaled_prediction = scaler.inverse_transform(prediction)

In [ ]:
# a=predictions.reshape(2562, 600)
# a.shape

In [ ]:
rescaled_prediction

In [ ]:
#Get dates for test data
time_series_array = test.index  

In [ ]:
# adding new no. of days
for k in range(0, future_days):
    time_series_array = time_series_array.append(time_series_array[-1:] + pd.DateOffset(1))

In [ ]:
df_forecast = pd.DataFrame(columns=["actual_confirmed","predicted"], index=time_series_array)

In [ ]:
test["confirmed"]

In [ ]:
df_forecast.loc[:,"predicted"] = rescaled_prediction[0:55]
df_forecast.loc[:,"actual_confirmed"] = test["confirmed"]

In [ ]:
df_forecast.plot(title="Predictions for next 10 days")
